In [1]:
import pandas as pd
from datasets import Dataset
from datasets import load_dataset
from langdetect import detect
from tqdm import tqdm 

In [2]:
# An empty list to store DataFrames for each language
dataframes = []

# Define the list of language configurations
languages = ['de', 'es', 'fr', 'ru', 'tu']

# Number of records to download for each language
records_to_download = 700

# Load the dataset for each language configuration
for lang in languages:
    print(f"Loading {records_to_download} records from MLSUM dataset for language: {lang}")
    dataset = load_dataset('mlsum', lang, split=f'train[:{records_to_download}]', trust_remote_code=True)

    # Convert to pandas DataFrame for easier manipulation
    df = pd.DataFrame(dataset)

    # Add a column for language
    df['language'] = lang

    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
data = pd.concat(dataframes, ignore_index=True)

# Filter to keep relevant columns
data = data.drop('url', axis=1, errors='ignore')

# Display the filtered DataFrame
print("Filtered Data with Language Detection:")
print(data.head(3))

Loading 700 records from MLSUM dataset for language: de
Loading 700 records from MLSUM dataset for language: es
Loading 700 records from MLSUM dataset for language: fr
Loading 700 records from MLSUM dataset for language: ru
Loading 700 records from MLSUM dataset for language: tu
Filtered Data with Language Detection:
                                                text  \
0  Transport im Viehwaggon, Fleischgeruch in der ...   
1  Marmorner Zebrastreifen, pompöse Gebäude: Sind...   
2  Wenn an diesem Montag die Landesvorsitzenden d...   

                                             summary    topic  \
0  Transport im Viehwaggon, Fleischgeruch in der ...  politik   
1  Marmorner Zebrastreifen, pompöse Gebäude: Sind...  politik   
2  Oskar Lafontaine gibt den Parteivorsitz der Li...  politik   

                                               title        date language  
0  So war Auschwitz: Erinnerungen einer Holocaust...  00/01/2010       de  
1  Kommunen in Not (3): Sindelfingen - Jens

In [3]:
# Create Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split dataset for training/evaluation (80% train, 20% validation)
dataset = dataset.train_test_split(test_size=0.2)

In [4]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Load translation model and tokenizer
translation_model_name = "facebook/mbart-large-50-many-to-one-mmt"
translation_model = MBartForConditionalGeneration.from_pretrained(translation_model_name)
translation_tokenizer = MBart50Tokenizer.from_pretrained(translation_model_name, src_lang="en_XX", tgt_lang="en_XX")

def translate_text(text):
    prompt = f"Translate the text to English: {text}"
    inputs = translation_tokenizer(prompt, return_tensors="pt", max_length=128, truncation=True)
    outputs = translation_model.generate(**inputs)
    translation = translation_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

2024-12-03 07:09:46.551813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 07:09:47.623313: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2024-12-03 07:09:47.623451: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [5]:
# Define the translation function to translate the summaries
def translate_summaries(dataset):
    translated_summaries = []
    for summary in tqdm(dataset["summary"], desc="Translating Summaries"):
        translated_summary = translate_text(summary)
        translated_summaries.append(translated_summary)
    return translated_summaries

# Translate the summaries in the dataset
translated_summaries_train = translate_summaries(dataset["train"])
translated_summaries_test = translate_summaries(dataset["test"])

# Now add the translated summaries as a new column
dataset["train"] = dataset["train"].add_column("translated_summary", translated_summaries_train)
dataset["test"] = dataset["test"].add_column("translated_summary", translated_summaries_test)

Translating Summaries: 100%|██████████| 700/700 [26:45<00:00,  2.29s/it]


Flattening the indices:   0%|          | 0/2800 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/700 [00:00<?, ? examples/s]

In [6]:
# Define the translation function
def translate_texts(texts, batch_size=16):
    """
    Translate a list of texts in batches.
    
    Args:
        texts (list): List of strings to translate.
        batch_size (int): Number of texts to translate at a time.

    Returns:
        list: Translated texts.
    """
    translated_texts = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating Input Texts"):
        batch = texts[i:i + batch_size]
        # Assume translate_text is a function that translates a single text
        translated_batch = [translate_text(text) for text in batch]
        translated_texts.extend(translated_batch)
    return translated_texts

# Translate the input texts in the train and test datasets
translated_texts_train = translate_texts(dataset["train"]["text"])
translated_texts_test = translate_texts(dataset["test"]["text"])

# Add the translated input texts as a new column
dataset["train"] = dataset["train"].add_column("translated_text", translated_texts_train)
dataset["test"] = dataset["test"].add_column("translated_text", translated_texts_test)

Translating Input Texts: 100%|██████████| 44/44 [1:26:11<00:00, 117.53s/it]


In [7]:
print(dataset["train"].column_names)

['text', 'summary', 'topic', 'title', 'date', 'language', 'translated_summary', 'translated_text']


In [8]:
def preprocess_data_for_summarization(examples):
    return {
        "input_text": [f"Summarize the following text: {text}" for text in examples["translated_text"]],
        "target_text": examples["translated_summary"]  # Use the translated summary here
    }

# Apply the preprocessing function to the dataset
summarization_dataset = dataset.map(preprocess_data_for_summarization, batched=True)

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [9]:
# (summarization_dataset["train"]["translated_summary"])

In [10]:
from peft import LoraConfig, get_peft_model, PeftType, TaskType
from transformers import TrainingArguments, Trainer, AutoModelForSeq2SeqLM, AutoTokenizer

# Load summarization model and tokenizer
summarization_model_name = "t5-small"  # Use T5 or any other Seq2Seq model
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q", "v"],  # LoRA applied to specific layers
    task_type=TaskType.SEQ_2_SEQ_LM
)

# Apply LoRA to the model
lora_model = get_peft_model(summarization_model, lora_config)

# Tokenize data for training
def tokenize_function(examples):
    inputs = summarization_tokenizer(
        examples["input_text"], max_length=512, truncation=True, padding="max_length", return_tensors="pt"
    )
    labels = summarization_tokenizer(
        examples["target_text"], max_length=128, truncation=True, padding="max_length", return_tensors="pt"
    )["input_ids"]
    inputs["labels"] = labels
    return inputs

tokenized_dataset = summarization_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

In [11]:
# tokenized_dataset["train"]['labels']

In [12]:
training_args = TrainingArguments(
    output_dir="./summarization_lora_results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",  # Save at the end of each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    load_best_model_at_end=True,  # Load the best model at the end of training
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=summarization_tokenizer,
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/tmp/ipykernel_129512/1657301929.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.310400,0.925675
2,0.924700,0.818257
3,0.837200,0.770432
4,0.818700,0.744333
5,0.834000,0.733954
6,0.798200,0.730913


TrainOutput(global_step=2100, training_loss=1.7337668827601842, metrics={'train_runtime': 4427.5871, 'train_samples_per_second': 3.794, 'train_steps_per_second': 0.474, 'total_flos': 2288962555084800.0, 'train_loss': 1.7337668827601842, 'epoch': 6.0})

In [26]:
def pipeline(input_text):
    
    # Translate text
    translated_text = translate_text(input_text)
    print("Translated text: ",translated_text)
    
    # Summarize translated text
    prompt = f"summarize this text: {translated_text}"
    inputs = summarization_tokenizer(
            prompt,
            return_tensors="pt",
            max_length=512,
            truncation=True
        )    
    # Generate summary
    outputs = lora_model.generate(
        **inputs,
        max_length=150,
        min_length=20,
        num_beams=4,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    summary = summarization_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [30]:
# Example usage
example_text = """Le soleil se levait doucement à l'horizon, projetant une lumière dorée sur les collines ondulantes. Les oiseaux commençaient à chanter, leur mélodie emplissant l'air frais du matin. Dans un petit village niché au pied des montagnes, les habitants s'affairaient déjà à leurs tâches quotidiennes. Les enfants couraient dans les ruelles pavées, riant et jouant à cache-cache, tandis que les adultes ouvraient les volets de leurs maisons en pierre.

Marie, une jeune femme aux cheveux bruns et aux yeux pétillants, sortit de sa maison en tenant un panier rempli de fleurs fraîches. Elle aimait commencer sa journée en cueillant des fleurs dans les champs voisins. Les couleurs vives des coquelicots, des marguerites et des bleuets illuminaient le paysage. Tandis qu'elle marchait, elle salua ses voisins avec un sourire chaleureux.

Au centre du village se trouvait une petite place animée où les commerçants installaient leurs étals. Le boulanger, avec sa toque blanche et ses mains enfarinées, sortait des baguettes croustillantes du four. L'odeur du pain chaud se mêlait aux arômes des fruits frais et des herbes vendues par les marchands. Les discussions animées des villageois créaient une ambiance joyeuse et conviviale.

Non loin de là, un vieux moulin à eau tournait lentement, alimenté par un ruisseau limpide. Les enfants aimaient s'y rassembler pour jeter des cailloux dans l'eau ou observer les poissons qui nageaient sous la surface. Les plus âgés racontaient des histoires sur les légendes locales, parlant de chevaliers courageux et de trésors cachés dans les montagnes environnantes.

À la lisière du village, un berger conduisait son troupeau de moutons vers les pâturages verdoyants. Le tintement des clochettes attachées aux cous des animaux résonnait dans l'air tranquille. Les chiens de berger, alertes et obéissants, veillaient à ce qu'aucun mouton ne s'éloigne. Le berger, avec son bâton en bois et son chapeau de paille, semblait en parfaite harmonie avec la nature.

Pendant ce temps, dans une petite ferme à la périphérie du village, un fermier et sa femme travaillaient dans leur potager. Ils plantaient des légumes et arrosaient les plantes sous le regard curieux d'un chat paresseux qui somnolait sur une"""
result = pipeline(example_text)
print("\nSummary:", result)

Translated text:  The sun rose gently on the horizon, casting a golden light over the rolling hills. The birds began to sing, their melody filling the fresh morning air. In a small village nestled at the foot of the mountains, the inhabitants were already busy with their daily tasks. Children ran through the cobbled streets, laughing and playing hide-and-seek, while adults opened the sides of their stone houses.

Summary: children ran through the cobbled streets, laughing and playing hide-and-seek, while adults opened the sides of their stone houses.


In [28]:
example_text = """वाशिंगटन - जैसा कि कांग्रेस इस सप्ताह सत्र समाप्त होने से पहले आखिरी गतिविधि के लिए लौटी है, उसे सरकारी शटडाउन को रोकने के लिए 20 दिसंबर की महत्वपूर्ण समय सीमा का सामना करना पड़ रहा है।

ऐसा प्रतीत होता है कि डेमोक्रेट और रिपब्लिकन ने एक सतत प्रस्ताव, या सीआर पारित करने के लिए इस्तीफा दे दिया है, जो 2025 की शुरुआत में सरकार को अस्थायी रूप से वित्त पोषित करेगा - संभवतः मार्च - क्योंकि उनके पास इस वर्ष पूर्ण वित्तपोषण सौदा करने के लिए समय नहीं बचा है। दोनों पार्टियाँ नए वित्तीय वर्ष के लिए समग्र व्यय स्तर पर भी सहमत नहीं हुई हैं, सरकार के विभिन्न हिस्सों में धन कैसे आवंटित किया जाए, इसकी तो बात ही छोड़ दें।


रिपब्लिकन के लिए, यह दोधारी तलवार है।

समय सीमा तय करने में रिपब्लिकन के लिए लाभ यह है कि नए साल में सरकारी फंडिंग को आकार देने के लिए उनके पास अधिक लाभ होगा, क्योंकि निर्वाचित राष्ट्रपति डोनाल्ड ट्रम्प व्हाइट हाउस में लौट आएंगे और जीओपी सीनेट पर नियंत्रण कर लेगी और एक संकीर्ण सदन बहुमत बनाए रखेगी।

बड़ा नकारात्मक पक्ष यह है कि इससे ट्रंप के राष्ट्रपति बनने की शुरुआत में ही एक महत्वपूर्ण समय सीमा तय हो जाएगी, जिससे संभावित रूप से सीनेट के माध्यम से उनके उम्मीदवारों की पुष्टि करने और रिपब्लिकन द्वारा उनके कर कटौती को बढ़ाने और उनके आव्रजन को आगे बढ़ाने के लिए बड़े पार्टी-लाइन बिल से मूल्यवान समय बर्बाद हो जाएगा। और सीमा सुरक्षा एजेंडा।

"हमें एक ही समय में बहुत सारी चीजें करनी हैं," हाउस मेजॉरिटी लीडर स्टीव स्कैलिस, आर-ला, ने ट्रम्प के दूसरे कार्यकाल के पहले 100 दिनों के बारे में कहा। "हम टहलने जा रहे हैं और गम चबा रहे हैं।"

कुछ रिपब्लिकन नए ट्रम्प राष्ट्रपति पद की शुरुआत में फंडिंग की समय सीमा में नहीं फंसना पसंद करेंगे।"""
result = pipeline(example_text)
print("\nSummary:", result)

Translated text:  Washington - As the Congress returns to its final activity before the session ends this week, it faces a crucial deadline of December 20 to stop the government shutdown. The Democrats and the Republicans seem to have resigned to pass a sustained motion, or CR, that will provide temporary financing to the government early in 2025 - possibly in March - because they have no time to make a full financing deal this year.

Summary: Democrats and the Republicans seem to have resigned to pass a sustained motion, or CR, that will provide temporary financing to the government early in 2025.


In [29]:
example_text = """واشنطن – مع عودة الكونجرس هذا الأسبوع للقيام بموجة أخيرة من النشاط قبل أن يختتم جلسته، فإنه يواجه موعدًا نهائيًا رئيسيًا في 20 ديسمبر لتجنب إغلاق الحكومة.

يبدو أن الديمقراطيين والجمهوريين مستسلمون لتمرير قرار مستمر، أو CR، من شأنه أن يمول الحكومة مؤقتًا حتى أوائل عام 2025 - على الأرجح في مارس - حيث ينفد الوقت أمامهم للتوصل إلى اتفاق تمويل كامل هذا العام. ولم يتفق الطرفان حتى على مستوى الإنفاق الإجمالي للسنة المالية الجديدة، ناهيك عن كيفية تخصيص الأموال عبر أجزاء من الحكومة.

ألمح زعيم الأغلبية في مجلس الشيوخ تشاك شومر، ديمقراطي من ولاية نيويورك، إلى حتمية مشروع قانون قصير الأجل يوم الاثنين، قائلاً: "نحن بحاجة إلى إبقاء الأحكام المثيرة للخلاف وغير الضرورية خارج أي تمديد للتمويل الحكومي، وإلا فسيكون من الصعب تمرير مشروع قانون". CR في الوقت المناسب.

وبالنسبة للجمهوريين، فإن هذا سيف ذو حدين.

الجانب الإيجابي بالنسبة للجمهوريين في تحديد الموعد النهائي هو أنهم سيكون لديهم المزيد من النفوذ لتشكيل التمويل الحكومي في العام الجديد، مع عودة الرئيس المنتخب دونالد ترامب إلى البيت الأبيض وسيطرة الحزب الجمهوري على مجلس الشيوخ والحفاظ على أغلبية ضيقة في مجلس النواب.

الجانب السلبي الكبير هو أنه سيحدد موعدًا نهائيًا حاسمًا في وقت مبكر من رئاسة ترامب، مما قد يستغرق وقتًا ثمينًا بعيدًا عن تأكيد مرشحيه من خلال مجلس الشيوخ ومن مشروع قانون الحزب الكبير الذي يتطلع إليه الجمهوريون لتمديد تخفيضاته الضريبية وتعزيز الهجرة. وأجندة أمن الحدود.

وأضاف كينيدي أنه من المرجح أن يعلق الكونجرس "30 [مليار دولار] إلى 40 مليار دولار من الإغاثة في حالات الكوارث" إلى الجمهورية التشيكية، بما في ذلك تمويل الولايات التي ضربتها الأعاصير هذا العام. وقال: "لن يكون ذلك كافيا، لكنه سيكون كافيا للبدء"."""

result = pipeline(example_text)
print("\nSummary:", result)

Translated text:  WASHINGTON, DC – With Congress returning this week to make one final move before it closes, it faces a major deadline on December 20 to avoid shutting down the government. Democrats and Republicans seem resigned to pushing through a decision, or CR, that would temporarily fund the government until early 2025 – probably in March – when they run out of time to agree on full funding this year.

Summary: Democrats and Republicans seem resigned to pushing through a decision that would temporarily fund the government until early 2025.


In [31]:
# Save the translation model and tokenizer
translation_model.save_pretrained("./translation_model")
translation_tokenizer.save_pretrained("./translation_model")

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'num_beams': 5, 'forced_bos_token_id': 250004}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./translation_model/tokenizer_config.json',
 './translation_model/special_tokens_map.json',
 './translation_model/sentencepiece.bpe.model',
 './translation_model/added_tokens.json')

In [14]:
lora_model.save_pretrained("./lora_summarization_model")
summarization_tokenizer.save_pretrained("./lora_summarization_model")